In [ ]:
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.losses import MeanSquaredError,BinaryCrossentropy,MeanAbsoluteError
import tensorflow as tf
import tensorflow_probability as tfp

In [ ]:
file_path = './train.csv'
file_path2 = './test.csv'

# Read the CSV file into a DataFrame
# train_data (+ val_data)
df = pd.read_csv(file_path)
df.index = df["id"]
df = pd.DataFrame(df.iloc[:, 1:])

# test_data
df2 = pd.read_csv(file_path2)
df2.index = df2["id"]
df2 = pd.DataFrame(df2.iloc[:,1:])
X_test = df2

# Split the data into training and validating sets
#X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2,random_state=42)

In [ ]:
df.columns

### eliminating outliers

In [ ]:
Q1 = df[['allelectrons_Total', 'density_Total', 'allelectrons_Average',
       'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
       'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
       'zaratio_Average', 'density_Average']].quantile(0.2)
Q3 = df[['allelectrons_Total', 'density_Total', 'allelectrons_Average',
       'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
       'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
       'zaratio_Average', 'density_Average']].quantile(0.8)
IQR = Q3 - Q1

# 'Column1'과 'Column3'에 대해서만 IQR 기반 이상치 제거
outlier_condition = ~((df[['allelectrons_Total', 'density_Total', 'allelectrons_Average',
       'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
       'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
       'zaratio_Average', 'density_Average']] < (Q1 - 1.5 * IQR)) | (df[['allelectrons_Total', 'density_Total', 'allelectrons_Average',
       'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
       'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
       'zaratio_Average', 'density_Average']] > (Q3 + 1.5 * IQR)))
df_no_outliers = df[outlier_condition.all(axis=1)]

# 결과 출력
#print("Original DataFrame:")
#print(df)
#print("\nDataFrame without Outliers:")
#print(df_no_outliers)
print(len(df_no_outliers))

In [ ]:
from matplotlib import pyplot as plt
df_no_outliers['Hardness'].plot(kind='hist', bins=20, title='val_e_Average')
plt.gca().spines[['top', 'right',]].set_visible(False)

###Building the model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, BatchNormalization, Dense
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.layers import Layer

class DropConnectLayer(Layer):
    def __init__(self, dropconnect_rate, **kwargs):
        super(DropConnectLayer, self).__init__(**kwargs)
        self.dropconnect_rate = dropconnect_rate

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=(input_shape[-1],), initializer="glorot_uniform", trainable=True)
        super(DropConnectLayer, self).build(input_shape)

    def call(self, inputs, training=None):
        if training:
            mask = tf.keras.backend.random_binomial(shape=tf.shape(self.kernel), p=1 - self.dropconnect_rate)
            return inputs * self.kernel * mask
        else:
            return inputs * self.kernel

    def compute_output_shape(self, input_shape):
        return input_shape

def build_model_with_dropconnect(input_shape, dropconnect_rate):
    model = Sequential([
        BatchNormalization(epsilon=1e-5, input_shape=(11,)),
        Dense(16, activation='relu', kernel_initializer=GlorotUniform()),
        #DropConnectLayer(dropconnect_rate),
        Dropout(0.5),
        Dense(32, activation='gelu'),
        #DropConnectLayer(dropconnect_rate),
        Dropout(0.5),
        Dense(64, activation='relu', kernel_initializer=GlorotUniform()),
        #DropConnectLayer(dropconnect_rate),
        Dropout(0.5),
        Dense(1)
    ])
    return model

input_shape = (11,)
dropconnect_rate = 0  # You can adjust this rate based on your needs
model = build_model_with_dropconnect(input_shape, dropconnect_rate)

#def build_model_using_sequential(input_shape):
#    model = Sequential([
#        BatchNormalization(epsilon=1e-5, input_shape=input_shape),
#        Dense(16, activation='relu', kernel_initializer=GlorotUniform()),
#        Dense(32, activation='gelu'),
#        Dense(64, activation='relu', kernel_initializer=GlorotUniform()),
#        Dense(1)
#    ])
#    return model

# Example of how to use the function with an input shape
input_shape = (11, )
model = build_model_with_dropconnect(input_shape, dropconnect_rate)

# Optionally, you can build the model to show the summary
#model.build((None, 11))
model.summary()

In [ ]:
def loss_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=50)

#def metric_fn(y_true, y_pred):
#    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=100) - tfp.stats.percentile(tf.abs(y_true - y_pred), q=0)

callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        patience=50,
        restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00001),
]

In [ ]:
num_folds = 5
batch_size = 64
num_epochs = 10

# Initialize KFold
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Iterate over folds
for fold, (train_indices, val_indices) in enumerate(kf.split(df_no_outliers)):
    print(f"Fold {fold + 1}/{num_folds}")

    # Split data into training and validation sets
    X_train_fold, X_val_fold = df_no_outliers.iloc[train_indices,:-1], df_no_outliers.iloc[val_indices, :-1]
    y_train_fold, y_val_fold = df_no_outliers.iloc[train_indices, -1], df_no_outliers.iloc[val_indices, -1]

    # Create an instance of your TensorFlow model
    #model = build_model_using_sequential(input_shape)  # Replace with how you create your model function
    model = build_model_with_dropconnect(input_shape, dropconnect_rate)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(0.013, beta_1=0.5), loss=loss_fn, metrics=['mae'])

    # Create and fit a TensorFlow Dataset for training
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train_fold.values, y_train_fold.values)).shuffle(len(X_train_fold)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val_fold.values, y_val_fold.values)).batch(batch_size)

    # Train the model
    model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

    # Evaluate the model on the validation set
    val_loss, val_mae = model.evaluate(val_dataset)
    print(f"Validation Loss: {val_loss}, Validation MAE: {val_mae}")

# After all folds, if you want to train on the full dataset for more epochs:
history = model.fit(
    df_no_outliers.iloc[:, :-1].values.astype('float32'),
    df_no_outliers.iloc[:, -1].values.astype('float32'),
    epochs=100,
    batch_size=64,
    callbacks=callbacks_list,
    validation_split=0.2
)

In [ ]:
y_pred = model.predict(X_test.astype('float32'))
df_prediction = pd.DataFrame(y_pred)
df_prediction

In [ ]:
from matplotlib import pyplot as plt
df_prediction[0].plot(kind='hist', bins=20, title=0)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
df_prediction[0].plot(kind='hist', bins=20, title=0)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
test= pd.read_csv('./sample_submission.csv')
test['Hardness'] = df_prediction
test.to_csv('submission.csv',index=False)

In [ ]:
plt.figure(figsize=(15,4))

plt.subplot(121)
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and validation losses')
plt.legend()

plt.subplot(122)
plt.plot(history.history['mae'], label='Training accuracy')
plt.plot(history.history['val_mae'], label='Validation accuracy')
plt.title('Training and validation accuracies')
plt.legend()